##Import df


In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [12]:
%cd '/content/drive/My Drive/Colab Notebooks'


/content/drive/My Drive/Colab Notebooks


In [45]:
#from google.colab import files
#uploaded = files.upload()

In [3]:
import io
import pandas as pd

#sub_df = pd.read_csv('sub_df.csv')

##Preprocessing


In [8]:
titles_df = sub_df["title"]
print(titles_df.head())

0                            GME sign from the QUAZAR.
1                          5th trade: GME Gang rise up
2                   How much to invest in GME and PLTR
3    Yesterday I was celebrating, today I am in the...
4    GME DD, look at all the out of stock stickers ...
Name: title, dtype: object


In [9]:
#lowercase, clean from characters
titles_df = titles_df.str.lower().str.replace(r'[^a-zA-Z\s]', '')
print(titles_df.head())


0                             gme sign from the quazar
1                            th trade gme gang rise up
2                   how much to invest in gme and pltr
3    yesterday i was celebrating today i am in the ...
4        gme dd look at all the out of stock stickers 
Name: title, dtype: object


In [4]:
import nltk
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [5]:
#stopwords
from nltk.corpus import stopwords

stop = stopwords.words('english')
#titles_df  = titles_df .apply(lambda x: " ".join(x for x in x.split() if x not in stop))
#print(titles_df.head())

In [63]:
#stemming DO NOT USE, use lemmatizing instead
#from nltk.tokenize import word_tokenize
#from nltk.stem import PorterStemmer

#stemmer = PorterStemmer()

#def stem_title(title): 
#  words = word_tokenize(title)
#  stemmed_words = [stemmer.stem(word) for word in words]
#  return " ".join(stemmed_words)

In [ ]:
#titles_df  = titles_df.apply(lambda x: stem_title(x))
#titles_df.head()

In [6]:
!python -m textblob.download_corpora

[nltk_data] Downloading package brown to /root/nltk_data...
[nltk_data]   Package brown is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package conll2000 to /root/nltk_data...
[nltk_data]   Package conll2000 is already up-to-date!
[nltk_data] Downloading package movie_reviews to /root/nltk_data...
[nltk_data]   Package movie_reviews is already up-to-date!
Finished.


In [7]:
#spell correct  & part-of-speech tagging 
from textblob import TextBlob
from nltk.tokenize import word_tokenize

def spellcorrect(title): #not necessary as a function, not used later
  title = TextBlob(title)
  return title.correct()

#lemmatization
from nltk.stem import WordNetLemmatizer 
from nltk.corpus import wordnet

lemmatizer = WordNetLemmatizer() 


def get_wordnet_pos(text):
  for tok, pos in nltk.pos_tag(word_tokenize(text)):
    if pos[0].lower() =='j':
      return wordnet.ADJ
    elif pos[0].lower() =='v':
      return wordnet.VERB
    elif pos[0].lower() =='n':
      return wordnet.NOUN
    elif pos[0].lower() =='r':
      return wordnet.ADV
    else:
      return wordnet.NOUN   #if anything else, just leave as noun

   
def tagandlemma(title): 
  words = word_tokenize(title)
  lemmatized_words = [lemmatizer.lemmatize(word, pos=f"{get_wordnet_pos(word)}") for word in words]
  return " ".join(lemmatized_words)


In [ ]:
titles_df  = titles_df.apply(lambda x: TextBlob(x).correct())  #takes waay too long leave out for now


In [ ]:
titles_df  = titles_df.apply(lambda x: tagandlemma(x))
print(titles_df.head())
#titles_df.to_csv('preproc_titles.csv')


In [ ]:
nltk.download('tagsets')
nltk.help.upenn_tagset()

##Word Vectors


In [8]:
!pip install spacy 
!python -m spacy download en_core_web_md

     |████████████████████████████████| 96.4MB 1.4MB/s 
  Created wheel for en-core-web-md: filename=en_core_web_md-2.2.5-cp37-none-any.whl size=98051305 sha256=865e14ae3b790567ca3e5e33dc6a3239f0aad8e7b28c61e32bfeeaa71a982c10
  Stored in directory: /tmp/pip-ephem-wheel-cache-b3o5emd5/wheels/df/94/ad/f5cf59224cea6b5686ac4fd1ad19c8a07bc026e13c36502d81
Successfully built en-core-web-md
  Found existing installation: en-core-web-md 2.2.0
    Uninstalling en-core-web-md-2.2.0:
      Successfully uninstalled en-core-web-md-2.2.0
✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_md')


In [19]:
from google.colab import files
uploaded = files.upload()

Saving reduced_titles_textblob_df1.csv to reduced_titles_textblob_df1.csv


In [13]:
import io
import pandas as pd

rdf = pd.read_csv('reduced_titles_textblob_df1.csv')
rdf = rdf.loc[:, ~rdf.columns.str.contains('^Unnamed')]
rdf.head()

,ptitle,polarity_textBlob
0,buy the dip the squeeze is in sight the hedge ...,1
1,what do i do gme,0
2,ignore all bots hold your gme retards,1
3,wsb gme rocket club black card,0
4,just for show gme attacks primed for headlines,0


In [14]:
#preproc
import re
rdf["ptitle"] = rdf["ptitle"].str.lower().str.replace(r'[^a-zA-Z\s]', '')
rdf["ptitle"] = rdf["ptitle"].apply(lambda x: " ".join(x for x in x.split() if x not in stop))
#rdf["ptitle"] = rdf["ptitle"].apply(lambda x: TextBlob(x).correct())  #wrong, makes weird tuple with letters
rdf["ptitle"] = rdf["ptitle"].apply(lambda x: tagandlemma(x))
rdf.head()


,ptitle,polarity_textBlob
0,buy dip squeeze sight hedge chopped ape strong...,1
1,gme,0
2,ignore bot hold gme retard,1
3,wsb gme rocket club black card,0
4,show gme attack prim headline,0


In [15]:
class Category: #sentiment 
  POSITIVE = "POSITIVE"
  NEUTRAL = "NEUTRAL"
  NEGATIVE = "NEGATIVE"

def convert_into_cat(sent):
  if sent==1: 
    return Category.POSITIVE
  elif sent==-1: 
    return Category.NEGATIVE
  else: 
    return Category.NEUTRAL
  

rdf["polarity_textBlob"] = rdf["polarity_textBlob"].apply(lambda x: convert_into_cat(x))
rdf["polarity_textBlob"].head()

0    POSITIVE
1     NEUTRAL
2    POSITIVE
3     NEUTRAL
4     NEUTRAL
Name: polarity_textBlob, dtype: object

In [36]:
type(rdf["polarity_textBlob"].iloc[0])

str

In [16]:
#split dataset 80 train 20 test 
train = rdf.iloc[0:40]
test = rdf.iloc[40:50]

train_x = train["ptitle"].tolist()
train_y = train["polarity_textBlob"].tolist()
train_y

['POSITIVE',
 'NEUTRAL',
 'POSITIVE',
 'NEUTRAL',
 'NEUTRAL',
 'NEUTRAL',
 'NEGATIVE',
 'NEGATIVE',
 'POSITIVE',
 'NEGATIVE',
 'POSITIVE',
 'POSITIVE',
 'NEUTRAL',
 'POSITIVE',
 'NEUTRAL',
 'NEUTRAL',
 'NEUTRAL',
 'POSITIVE',
 'POSITIVE',
 'POSITIVE',
 'NEGATIVE',
 'POSITIVE',
 'POSITIVE',
 'NEUTRAL',
 'NEUTRAL',
 'NEGATIVE',
 'NEUTRAL',
 'NEUTRAL',
 'POSITIVE',
 'NEUTRAL',
 'POSITIVE',
 'POSITIVE',
 'POSITIVE',
 'NEGATIVE',
 'POSITIVE',
 'POSITIVE',
 'NEUTRAL',
 'NEGATIVE',
 'NEUTRAL',
 'POSITIVE']

In [17]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer(binary=True, ngram_range=(1,2))
train_x_vectors = vectorizer.fit_transform(train_x)
print(vectorizer.get_feature_names())
print(train_x_vectors.toarray())

['able', 'able sell', 'add', 'add liquidity', 'allow', 'allow gme', 'allow large', 'also', 'also start', 'amc', 'amc bb', 'amc gme', 'amc low', 'ameritrade', 'ameritrade say', 'amp', 'amp amc', 'analysis', 'analysis jul', 'ape', 'ape strong', 'asetniop', 'astound', 'astound manipulation', 'attack', 'attack prim', 'bb', 'bb vv', 'benefit', 'benefit market', 'black', 'black card', 'block', 'block amc', 'bot', 'bot hold', 'bought', 'bought start', 'buy', 'buy buy', 'buy dip', 'buy gme', 'buyer', 'buyer short', 'buying', 'buying gme', 'card', 'cent', 'cent squeeze', 'check', 'check european', 'chopped', 'chopped ape', 'club', 'club black', 'consider', 'consider sell', 'creator', 'creator asetniop', 'credit', 'credit creator', 'deep', 'deep gme', 'dfvs', 'dfvs initial', 'dip', 'dip squeeze', 'downside', 'downside please', 'dumbass', 'dumbass buy', 'eli', 'endjan', 'endjan price', 'etoro', 'etoro force', 'european', 'european fella', 'every', 'every gme', 'everybody', 'everybody flood', 'eve

In [39]:
!pip3 install https://github.com/explosion/spacy-models/releases/download/en_core_web_md-2.2.0/en_core_web_md-2.2.0.tar.gz


     |████████████████████████████████| 96.4MB 100kB/s 
  Created wheel for en-core-web-md: filename=en_core_web_md-2.2.0-cp37-none-any.whl size=98072934 sha256=d7359eaccd9fc1711b4b8a476db19b3c3c56217509cbfebbef912d630e4fd5c4
  Stored in directory: /root/.cache/pip/wheels/5f/3e/c9/36dd6e13b449fd84cd1f94b72dfbc559daf09f53dbf4e697a3
Successfully built en-core-web-md
  Found existing installation: en-core-web-md 2.2.5
    Uninstalling en-core-web-md-2.2.5:
      Successfully uninstalled en-core-web-md-2.2.5


In [18]:
import spacy
nlp = spacy.load("en_core_web_sm")

In [19]:
docs = [nlp(title) for title in train_x]
train_x_word_vectors = [x.vector for x in docs]

In [20]:
from sklearn import svm

clf_svm_wv = svm.SVC(kernel='linear')
clf_svm_wv.fit(train_x_word_vectors, train_y)


SVC(C=1.0, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='linear',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

In [22]:
pred = []
for eachtitle in range(len(test.index)):
  test_x = [test.iloc[eachtitle]["ptitle"]]
  test_docs = [nlp(text) for text in test_x]
  test_x_word_vectors = [x.vector for x in test_docs]
  pred.append(clf_svm_wv.predict(test_x_word_vectors))
pred

[array(['POSITIVE'], dtype='<U8'),
 array(['POSITIVE'], dtype='<U8'),
 array(['NEGATIVE'], dtype='<U8'),
 array(['NEUTRAL'], dtype='<U8'),
 array(['POSITIVE'], dtype='<U8'),
 array(['NEGATIVE'], dtype='<U8'),
 array(['POSITIVE'], dtype='<U8'),
 array(['POSITIVE'], dtype='<U8'),
 array(['POSITIVE'], dtype='<U8'),
 array(['POSITIVE'], dtype='<U8')]

In [25]:
nonmatched_predictions = pd.DataFrame(columns=["title", "orig_sent", "pred"])
orig_sent = test["polarity_textBlob"].tolist()
orig_title = test["ptitle"].tolist()
pred_l = [pred[i][0] for i in range(len(pred))]

for i in range(len(orig_sent)): 
    if pred_l[i]!=orig_sent[i]:
      nonmatched_predictions = nonmatched_predictions.append({"title": orig_title[i],"orig_sent": orig_sent[i], "pred" : pred_l[i]}, ignore_index=True)
nonmatched_predictions

,title,orig_sent,pred
0,broker good way buy gme,NEUTRAL,POSITIVE
1,wsb gme bull poem,POSITIVE,NEGATIVE
2,say lord give sign gme go ok lord say look pho...,POSITIVE,NEGATIVE
3,sure already rh allows buying additional share...,NEUTRAL,POSITIVE
4,break announcement sec suspicious activity gme...,NEGATIVE,POSITIVE
